In [95]:
import numpy as np
import xarray as xr
import pandas as pd
import math
import matplotlib.pyplot as plt
import nc_time_axis
import cftime

from palettable.cmocean.diverging import Balance_10
from pathlib import Path
from datetime import datetime
from datetime import timedelta

In [96]:
def dy_to_cftime(dy):
    '''
    A horrible hacky way to try and convert decimal years (yyyy.yyy)
    to a datetime that will cope with dates beyond 2622 (pandas limitation
    of 64-bit integer for their date range), in this case cftime.
    
    Currently assuming a 365-day calendar, as we are resampling annually,
    only the year is important.
    
    If anyone can improve on this, please do!
    '''
    
    # Split into year and decimal part of year
    dec, year = math.modf(dy)

    # Create a numpy datetime for the year
    dt = datetime(int(year), 1, 1)

    # Convert decimal year to days - this is assuming 365 day calendar
    days = dec * 365
    
    # Use timedelta to add days to year
    td = timedelta(days=days)
    dt = dt + td

    # Create cftime datetime
    cf_dt = cftime.datetime(dt.year, dt.month, dt.day)
    
    return cf_dt
    

In [97]:
def streamfunction(ds):
    '''
    From the Dataset containing the transport in Sv/10m, 
    calculate the streamfunction as the integral sum from the
    surface downwards, take the annual and time series means,
    and the maximum streamfunction for each time and latitude.
    '''
    
    # Convert time coordinates from decimal year to cf datetime, to allow resampling by
    # year and also cope with years beyond 2262 (pandas datetime cannot handle these).
    dt = []

    for t in ds.time.values:
        dt.append(dy_to_cftime(t))

    dt = np.array(dt)
    
    # Add the new datetime to the Dataset and swap 'time' (decimal year) and 'datetime' (cftime)
    # as dimensions
    ds['datetime'] = (('time'), dt)
    transport_ds = ds.swap_dims({'time': 'datetime'})
    
    # Calculate streamfunction as cumulative integration from the surface downward
    strf_ds = transport_ds.cumsum(dim='depth')
    
    # Calculate annual mean by resampling by year
    avg_strf_ds = strf_ds.resample(datetime='Y').mean()
    
    # Take the historical (timeseries) mean to plot the overturning
    hist_strf_ds = strf_ds.mean(dim='datetime')
    
    # Get maximum of annual streamfunction as the AMOC index
    max_strf_ds = avg_strf_ds.max(dim='depth')
    
    return hist_strf_ds, max_strf_ds

In [98]:
def plot_overturning(hist_ds, model):
    '''
    Plot the historical mean overturning streamfunction.
    '''
    
    plt.figure()
    plt.contourf(hist_ds.latitude, hist_ds.depth, hist_ds.transport.transpose(), levels=13, cmap=Balance_10.mpl_colormap)
    plt.title('Historical mean streamfunction for ' + model)
    plt.tight_layout()
    plt.gca().invert_yaxis()
    plt.xlabel('Latitude')
    plt.ylabel('Depth')
    plt.savefig('/home/users/elw2u16/hackathon/project04/results/amoc_streamfunction/' + model + '.png', format='png', dpi=200, bbox_inches='tight')
    plt.close()

In [99]:
def plot_amoc(max_ds, model, lat):
    '''
    Plot the maximum overturning streamfunction at a given latitude
    '''
    
    # Extract data at given latitude
    lat_max_ds = max_ds.sel(latitude=lat)
    
    # Convert cftime to plottable type
    c_d_time = [nc_time_axis.CalendarDateTime(item, "365_day") for item in lat_max_ds.datetime.values]
    
    plt.figure()
    plt.plot(c_d_time, lat_max_ds.transport)
    plt.ylabel('AMOC [Sv]')
    plt.xlim([c_d_time[0], c_d_time[-1]])
    plt.title('AMOC strength at latitude ' + str(lat) + 'N for ' + model)
    plt.tight_layout()
    plt.savefig('/home/users/elw2u16/hackathon/project04/results/amoc_index_timeseries/' + model + '_' + str(lat) + '.png', format='png', dpi=200, bbox_inches='tight')
    plt.close()

In [100]:
def save_amoc_index(ds, lat, mip, model_name):
    '''
    Create a netCDF file for the maximum overturning streamfunction
    as an AMOC index for the given model/scenario/forcing with
    appropriate metadata.
    '''
    
    # Extract data at given latitude
    lat_ds = ds.sel(latitude=lat)
    
    # Save to local directory as cannot save to group workspace from notebook
    home_dir = str(Path.home())
    results_dir = home_dir + '/hackathon/project04/data/processed_data/CMIP6/amoc/'
    
    # Get scenario from model name to save in correct directory
    model, scenario, forcing = model_name.split('_')
    
    # Get today's date as formatted string
    today = np.datetime64('now')
    ts = pd.to_datetime(str(today))
    
    # Add metadata
    lat_ds.attrs['short_desc'] = 'AMOC index (annual mean max streamfunction)'
    lat_ds.attrs['latitude'] = str(lat)
    lat_ds.attrs['model'] = model
    lat_ds.attrs['scenario'] = scenario
    lat_ds.attrs['forcing'] = forcing
    lat_ds.attrs['date_created'] = ts.strftime('%d %b %Y')
    
    # Save netcdf
    lat_ds.to_netcdf(results_dir + mip + '/' + scenario + '/' + model_name + '-1ym-amoc-index_' + str(lat) + '.nc')
    

In [101]:
def save_10yr_mean_amoc_index(ds, lat, mip, model_name):
    '''
    Create 10-year rolling mean of AMOC index.
    '''
    
    # Extract data at given latitude
    lat_ds = ds.sel(latitude=lat)
    
    # Take rolling 10-year mean
    mean_ds = lat_ds.rolling(datetime=10).mean()
    
    # Save to local directory as cannot save to group workspace from notebook
    home_dir = str(Path.home())
    results_dir = home_dir + '/hackathon/project04/data/processed_data/CMIP6/amoc/'
    
    # Get scenario from model name to save in correct directory
    model, scenario, forcing = model_name.split('_')
    
    # Get today's date as formatted string
    today = np.datetime64('now')
    ts = pd.to_datetime(str(today))
    
    # Add metadata
    mean_ds.attrs['short_desc'] = 'AMOC index (10-year rolling mean max streamfunction)'
    mean_ds.attrs['latitude'] = str(lat)
    mean_ds.attrs['model'] = model
    mean_ds.attrs['scenario'] = scenario
    mean_ds.attrs['forcing'] = forcing
    mean_ds.attrs['date_created'] = ts.strftime('%d %b %Y')
    
    # Save netCDF 
    mean_ds.to_netcdf(results_dir + mip + '/' + scenario + '/' + model_name + '-10ym-amoc-index_' + str(lat) + '.nc')
    
    

In [102]:
def process_group(scenario_group, mip, scenario, lat):
    '''
    Do all the processing required for each model scenario.
    
    Get the AMOC index (maximum streamfunction) and the historical mean streamfunction,
    save the AMOC index and its 10-year rolling mean, and
    plot the AMOC index and the overturning streamfunction.
    '''
    
    data_dir = '/gws/pw/j05/cop26_hackathons/bristol/project04/raw_data/CMIP6/amoc/'
    group_dir = data_dir + mip + '/' + scenario + '/'
    
    for i in np.arange(scenario_group.scenario.size):
    
        model = scenario_group.iloc[i]

        # Load transport data
        model_name = model.model + '_' + model.scenario + '_' + model.forcing
        print('Processing', group_dir + model_name + '_gn_amoc10m.nc')
        transport_ds = xr.open_dataset(group_dir + model.model + '_' + model.scenario + '_' + model.forcing + '_gn_amoc10m.nc', use_cftime=True)
        
        # Get the maximum and historical mean streamfunction
        hist_strf_ds, max_strf_ds = streamfunction(transport_ds)
        
        # Save the maximum streamfunction as AMOC index
        save_amoc_index(max_strf_ds, lat, mip, model_name)
        
        # Save 10-year rolling mean of AMOC index
        save_10yr_mean_amoc_index(max_strf_ds, lat, mip, model_name)

        # Get first part of filename to save figure                                                
        plot_amoc(max_strf_ds, model_name, lat) 
        
        # Plot overturning streamfunction
        plot_overturning(hist_strf_ds, model_name)

In [103]:
# Get primary AMOC list
home_dir = str(Path.home())
model_list = home_dir + '/hackathon/project04/working_amoc_list'
amoc_models = pd.read_csv(model_list, names=['model', 'scenario', 'forcing'], delimiter=',', skipinitialspace=True)

In [104]:
# Split models by scenario
scenario_group = amoc_models.groupby('scenario')

ssp126 = scenario_group.get_group('ssp126')
ssp245 = scenario_group.get_group('ssp245')
ssp585 = scenario_group.get_group('ssp585')
historical = scenario_group.get_group('historical')

In [107]:
# Do the processing for each model + scenario for 26.5 N

process_group(historical, 'CMIP', 'historical', 26.5)
# process_group(ssp126, 'ScenarioMIP', 'ssp126', 26.5)
# process_group(ssp245, 'ScenarioMIP', 'ssp245', 26.5)
# process_group(ssp585, 'ScenarioMIP', 'ssp585', 26.5)

Processing /gws/pw/j05/cop26_hackathons/bristol/project04/raw_data/CMIP6/amoc/CMIP/historical/ACCESS-CM2_historical_r1i1p1f1_gn_amoc10m.nc
Processing /gws/pw/j05/cop26_hackathons/bristol/project04/raw_data/CMIP6/amoc/CMIP/historical/ACCESS-ESM1-5_historical_r1i1p1f1_gn_amoc10m.nc
Processing /gws/pw/j05/cop26_hackathons/bristol/project04/raw_data/CMIP6/amoc/CMIP/historical/BCC-CSM2-MR_historical_r1i1p1f1_gn_amoc10m.nc
Processing /gws/pw/j05/cop26_hackathons/bristol/project04/raw_data/CMIP6/amoc/CMIP/historical/BCC-ESM1_historical_r1i1p1f1_gn_amoc10m.nc
Processing /gws/pw/j05/cop26_hackathons/bristol/project04/raw_data/CMIP6/amoc/CMIP/historical/CAMS-CSM1-0_historical_r1i1p1f1_gn_amoc10m.nc
Processing /gws/pw/j05/cop26_hackathons/bristol/project04/raw_data/CMIP6/amoc/CMIP/historical/CESM2_historical_r1i1p1f1_gn_amoc10m.nc
Processing /gws/pw/j05/cop26_hackathons/bristol/project04/raw_data/CMIP6/amoc/CMIP/historical/CESM2-FV2_historical_r1i1p1f1_gn_amoc10m.nc
Processing /gws/pw/j05/cop26_h

FileNotFoundError: [Errno 2] No such file or directory: b'/gws/pw/j05/cop26_hackathons/bristol/project04/raw_data/CMIP6/amoc/CMIP/historical/NorCPM1_historical_r1i1p1f1_gn_amoc10m.nc'